Macros 



scm> ('(+ 1 2))
Traceback (most recent call last):
 0	((quote (+ 1 2)))
Error: pair is not callable: (+ 1 2)
scm> '(+ 1 2)
(+ 1 2)
scm> (eval '(+ 1 2))
3 
scm> (list '/ 10 2)
(/ 10 2) 
scm> (eval (list '/ 10 2))
5 
scm> (define x 2)
x 
scm> (list '+ x 2)
(+ 2 2) 
scm> (list '(+ x 2))
((+ x 2)) 
scm> (list '+ 'x 2)
(+ x 2) 
scm> (eval (list '(+ x 2)))
Traceback (most recent call last):
 0	(eval (list (quote (+ x 2))))
 1	((+ x 2))
 Error: int is not callable: 4
scm> (eval (list '+ 'x 2))
4
scm> (list / 10 2)
(#[/] 10 2)
scm> (eval (list / 10 2))
Traceback (most recent call last):
 0	(eval (list / 10 2))
 1	(#[/] 10 2)
 2	#[/]
Error: malformed list: #[/]

Begin 不是局部函数，对全局有影响
多个表达式依次求值，返回最后一个

scm>(begin 3 2 1)
1
scm> (begin (define x 2) (define x (+ x 1)) x)
3


Let 先绑定symbol，再运行func body
(let ((symbol1 expr1); let内部的临时绑定
      (symbol2 expr2) ;这里的所有绑定顺序是并行的，假定每个相互独立
…)
body)

scm> (let ((x 2)
(y 3))
(+ x y))
5

define-macro
传入参数表达式时不求值,把表达式传入body变换后再求值

传入一个expr 把expr改写成函数体的样子，然后eval

scm> (define-macro (twice expr)
     (list 'begin expr expr))
twice
scm> (twice (print 2))
2
2
scm> (begin (print 2) (print 2))
2
2


scm>(define-macro (add-to x value)
    (list 'define 'x (list '+ (eval  value) x)))   
add-to
scm> (add-to x ( + 1 2))
x
scm> x
4

scm> (define-macro (add-to var expr)
...> (list 'define var (list '+ var expr)))
add-to

scm> (add-to 2 (+ 4 5))
Traceback (most recent call last):
 0	(add-to 2 (+ 4 5))
 1	(define 2 (+ 2 (+ 4 5)))
Error: non-symbol: 2
scm> (define x 2)
x
scm> (add-to x (+ 4 5))
x
scm> x
11



Map
(map <func> <list>)
***return a list,which apply func in every elem in list***


scm> (for x in '(1 2 3 4) do (* x x))
(1 4 9 16)
scm> (map (lambda (x) (* x x)) '(1 2 3 4))
(1 4 9 16)

(define-macro (for sym in vals do expr)
(list 'map (list 'lambda (list sym) expr) vals))
***;for is real name,"in" and "do" just take place***
***return value : (  map (lambda(sym) <expr>) vals  ) ***

Quasi-Quotation

对部分表达式求值，部分表达式保留的简写方法

(define-macro (for sym vals expr)
(list 'map (list 'lambda (list sym) expr) vals))
等价于
(define-macro (for sym vals expr)
`(map (lambda (,sym) ,expr) ,vals))
 *** ,求值   '全部保留    `非,保留 ***

scm> (define x 3)
x
scm> '(1 ,x 3)
(1 (unquote x) 3)

短路


scm> (define-macro (f x) (car x))
f
scm> (f (+ 2 3))
#[+]
scm>  (define x 2000)
x
scm> (f (x y z))
2000
scm> y
Traceback (most recent call last):
scm> y	
Error: unknown identifier: y

